1. **Data Preprocessing**
   - Data loading and preprocessing (e.g., normalization, resizing, augmentation).
   - Create visualizations of some images, and labels.

In [ ]:
# 1. Data Preprocessing
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("GPU Devices: ", tf.config.experimental.list_physical_devices('GPU'))
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization, Dropout

# Cargar CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


In [ ]:
# Visualizar muestras
num_classes = 10
samples_per_class = 10
fig, axes = plt.subplots(num_classes, samples_per_class, figsize=(15, 15))

for i in range(num_classes):
    class_indices = np.where(y_train == i)[0]
    random_indices = np.random.choice(class_indices, samples_per_class, replace=False)
    for j, idx in enumerate(random_indices):
        ax = axes[i, j]
        ax.imshow(x_train[idx])
        ax.axis('off')

plt.show()

In [7]:
# Preprocesamiento
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

2. **Model Architecture**
   - Design a CNN architecture suitable for image classification.
   - Include convolutional layers, pooling layers, and fully connected layers.

In [8]:
# Your code here :
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
# Add more Dropout layers and L2 Regularization

from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler

model = Sequential([
    Conv2D(64, (3,3), padding='same', activation='relu', input_shape=(32,32,3), kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Conv2D(64, (3,3), padding='same', activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.4),
    
    Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Conv2D(128, (3,3), padding='same', activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.5),
    
    Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Conv2D(256, (3,3), padding='same', activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.5),
    
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
])


3. **Model Training**
   - Train the CNN model using appropriate optimization techniques (e.g., stochastic gradient descent, Adam).
   - Utilize techniques such as early stopping to prevent overfitting.

In [ ]:
# Use Learning Rate Scheduler

def lr_schedule(epoch, lr):
    if epoch > 75:
        lr = 0.0005
    if epoch > 100:
        lr = 0.0003
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# Add Early Stopping and Reduce Learning Rate on Plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)

# Fit the model with the new data generator and learning rate scheduler

history = model.fit(x_train, y_train, batch_size=512,
                   epochs=150,
                   validation_data=(x_test, y_test),
                   callbacks=[early_stopping, reduce_lr, lr_scheduler])

#plot loss
plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='red', label='val')


# plot accuracy
plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='green', label='train')
plt.plot(history.history['val_accuracy'], color='red', label='val')
plt.show()

4. **Model Evaluation**
   - Evaluate the trained model on a separate validation set.
   - Compute and report metrics such as accuracy, precision, recall, and F1-score.
   - Visualize the confusion matrix to understand model performance across different classes.

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

# Predicciones
predictions = model.predict(x_test)
predictions_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Asegurarse de que las clases estén en el rango correcto (0-9 para CIFAR-10)
print("Rango de predicciones:", np.min(predictions_classes), "-", np.max(predictions_classes))
print("Rango de valores reales:", np.min(true_classes), "-", np.max(true_classes))

# Calcular métricas solo si los rangos son correctos
if (0 <= np.min(predictions_classes) <= 9) and (0 <= np.max(predictions_classes) <= 9):
    print("\nMatriz de Confusión:")
    print(confusion_matrix(true_classes, predictions_classes))

    print("\nMétricas de Evaluación:")
    print(f"F1-Score: {f1_score(true_classes, predictions_classes, average='weighted'):.4f}")
    print(f"Precisión: {precision_score(true_classes, predictions_classes, average='weighted'):.4f}")
    print(f"Recall: {recall_score(true_classes, predictions_classes, average='weighted'):.4f}")
else:
    print("Error: Las predicciones no están en el rango esperado")

In [ ]:
model.save('saved_models/img-class-cnn.keras')

5. **Transfer Learning**
    - Evaluate the accuracy of your model on a pre-trained models like ImagNet, VGG16, Inception... (pick one an justify your choice)
        - You may find this [link](https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub) helpful.
        - [This](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html) is the Pytorch version.
    - Perform transfer learning with your chosen pre-trained models i.e., you will probably try a few and choose the best one.

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

# Cargar VGG16 pre-entrenado
base_model = VGG16(weights='imagenet', 
                   include_top=False, 
                   input_shape=(32, 32, 3))
                   #classifier_activation="SoftMax"))

# Congelar capas base
for layer in base_model.layers:
    layer.trainable = False

# Crear modelo de transfer learning
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

transfer_model = Model(inputs=base_model.input, outputs=predictions)

# Entrenar modelo de transfer learning
transfer_model.compile(optimizer=Adam(learning_rate=0.001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

history_transfer = transfer_model.fit(x_train, 
                                    y_train,
                                    batch_size=128,
                                    epochs=50,
                                    validation_data=(x_test, y_test),
                                    callbacks=[early_stopping, reduce_lr])

# Evaluar métricas después del transfer learning
predictions = transfer_model.predict(x_test)
predictions_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

print("\nMétricas después del transfer learning:")
print(f"F1-Score: {f1_score(true_classes, predictions_classes, average='weighted'):.4f}")
print(f"Precisión: {precision_score(true_classes, predictions_classes, average='weighted'):.4f}")
print(f"Recall: {recall_score(true_classes, predictions_classes, average='weighted'):.4f}")

#plot loss
plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history_transfer.history['loss'], color='blue', label='train')
plt.plot(history_transfer.history['val_loss'], color='red', label='val')


# plot accuracy
plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history_transfer.history['accuracy'], color='green', label='train')
plt.plot(history_transfer.history['val_accuracy'], color='red', label='val')
plt.show()

In [ ]:
model.save('saved_models/img-class-cnn-transer.keras')

#Comment part 2 of step 5

In [ ]:
# Fine-tuning
for layer in base_model.layers[-4:]:
    layer.trainable = True

transfer_model.compile(optimizer=Adam(learning_rate=0.0001),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

history_fine_tuning = transfer_model.fit(x_train, 
                                       y_train,
                                       batch_size=128,
                                       epochs=30,
                                       validation_data=(x_test, y_test),
                                       callbacks=[early_stopping, reduce_lr])

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

# Predicciones
predictions = transfer_model.predict(x_test)  # Nota: cambiado de base_model a transfer_model
predictions_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Asegurarse de que las clases estén en el rango correcto (0-9 para CIFAR-10)
print("Rango de predicciones:", np.min(predictions_classes), "-", np.max(predictions_classes))
print("Rango de valores reales:", np.min(true_classes), "-", np.max(true_classes))

# Calcular métricas solo si los rangos son correctos
if (0 <= np.min(predictions_classes) <= 9) and (0 <= np.max(predictions_classes) <= 9):
    print("\nMatriz de Confusión:")
    print(confusion_matrix(true_classes, predictions_classes))

    print("\nMétricas de Evaluación:")
    print(f"F1-Score: {f1_score(true_classes, predictions_classes, average='weighted'):.4f}")
    print(f"Precisión: {precision_score(true_classes, predictions_classes, average='weighted'):.4f}")
    print(f"Recall: {recall_score(true_classes, predictions_classes, average='weighted'):.4f}")
else:
    print("Error: Las predicciones no están en el rango esperado")

#

In [ ]:
# Your code here :
#plot loss
plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history_fine_tuning.history['loss'], color='blue', label='train')
plt.plot(history_fine_tuning.history['val_loss'], color='red', label='val')


# plot accuracy
plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history_fine_tuning.history['accuracy'], color='green', label='train')
plt.plot(history_fine_tuning.history['val_accuracy'], color='red', label='val')
plt.show()

In [ ]:
model.save('saved_models/img-class-cnn-fine.keras')